1st submission 30 epoch took 4+ hours

* Mount google drive so that files can be read and written to


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* Download and import indicnlp which is used for hindi text preprocessing

In [2]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

from indicnlp import loader
loader.load()

from indicnlp.normalize.indic_normalize import BaseNormalizer

from indicnlp.tokenize import indic_tokenize  

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1271, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1271 (delta 50), reused 54 (delta 25), pack-reused 1178
Receiving objects: 100% (1271/1271), 9.56 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (654/654), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 41.82 MiB/s, done.
Resolving deltas: 100% (51/51), done.


*   Import all necessary packages including pytorch
*   csv, string, random are python inbuilt libraries.
*   csv is used to read the csv into a 2d array
*   string is used to do string processing on the text 
*   random is used to generate pseudo-random number or pick something pseudo-randomly.
*   pytorch is used for the sequence to sequence model.


In [3]:
import csv
import string
import random

import torch
import torch.nn as nn

'\nSEED = 777\n\nrandom.seed(SEED)\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\ntorch.cuda.manual_seed(SEED)\ntorch.backends.cudnn.benchmark = True'


* Path to train.csv which contains training set --> traincsv


In [4]:
traincsv = "drive/MyDrive/AssignmentNLP/train.csv"



* Store training data in 2d list --> data 

In [5]:
with open(traincsv, newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

print(len(data))
data = data[1:]

102323




* Remove all punctuations from hindi and english text
* Remove empty strings
* remove all sentences from hindi column which has only english alphabets or numbers



In [6]:
translate_table = dict((ord(char), None) for char in string.punctuation)
c = 0
row = 0
while(row<len(data)):
    # print(row)
    texts_hi = str(data[row][1]).translate(translate_table).strip().lower()
    texts_en = str(data[row][2]).translate(translate_table).strip().lower()
    
    if texts_hi=='' or texts_en=='' or texts_hi in string.punctuation or texts_en in string.punctuation or len(texts_hi)>100:
        c+=1
        data.pop(row)
        
    else:
        data[row][1] = texts_hi  # texts has no punctuations
        data[row][2] = texts_en

        tmp = texts_hi.replace(" ","")
        cnt = 0
        for i in tmp:
            if (i>='a' and i<='z') or i.isnumeric():
                cnt+=1
        if cnt == len(tmp):
            data.pop(row)
            c+=1
        else:
            row+=1

print(c, len(data))

13128 89194


* Normalize hindi texts (I can't read hindi or understand hindi grammar and spellings so I didn't understand what normalization exactly does but seeing the documentation, I thought it helps make the spellings and text consistent)

In [7]:
normalizer = BaseNormalizer("hi", remove_nuktas=False)

row = 0
while(row<len(data)):
    data[row][1]=normalizer.normalize(str(data[row][1]))
    row+=1


*   Tokenize hindi and english (both using indicnlp)
*   Only add tokens that are present 3 times atleast in the dataset.
*   Build vocabulary of hindi and english text with tokens present atleast twice in training data using 4 dictionaries:

> *   English token to unique index --> eng_vocab_stoi
*   Unique index to english token   --> eng_vocab_itos
*   Hindi token to unique index     --> hin_vocab_stoi
*   Unique index to hindi token     --> hin_vocab_itos



*   Next create training data (2D list having senteces represented by indices) for hindi and english --> hin_train, eng_train 
*   In the training data, sentences are sorted primarily according to the length of hindi sentences, and secondarily according to length of english sentences (so that less padding is required later during creation of batches)
*   Convert the tokens in the sentences to their respective indices
*   Put special index 0 (representing unknown) for tokens which are not present in vocabulary (because they are seen only once)
*   Append initial and end tokens to all hindi and english sentences



In [9]:
eng_vocab_itos = {1:"<pad>", 2:"<sos>", 3:"<eos>", 0:"<ukn>"}
eng_vocab_stoi = {"<pad>":1, "<sos>":2, "<eos>":3, "<ukn>":0}
hin_vocab_itos = {1:"<pad>", 2:"<sos>", 3:"<eos>", 0:"<ukn>"}
hin_vocab_stoi = {"<pad>":1, "<sos>":2, "<eos>":3, "<ukn>":0}
eng_seen=set()
hin_seen=set()
tokenized = []
eng_train = []
hin_train = []
eng_count = 4
hin_count = 4

def CreateVocab(tokens, seen, vocab_stoi, vocab_itos, count):
    for wrd in tokens:
        if wrd not in seen:
            seen.add(wrd)
        elif wrd not in vocab_stoi:
            vocab_stoi[wrd] = count
            vocab_itos[count] = wrd
            count+=1
    return count

def CreateTrain(tokens, vocab_stoi):
    sentence = [2]
    for wrd in tokens:
        if wrd in vocab_stoi:
            sentence.append(vocab_stoi[wrd])
        else:
            sentence.append(0)
    sentence.append(3)
    return sentence


for i in data:
    eng_tokens = indic_tokenize.trivial_tokenize(i[2])
    hin_tokens = indic_tokenize.trivial_tokenize(i[1])
    tokenized.append([hin_tokens, eng_tokens])
    eng_count = CreateVocab(eng_tokens, eng_seen, eng_vocab_stoi, eng_vocab_itos, eng_count)
    hin_count = CreateVocab(hin_tokens, hin_seen, hin_vocab_stoi, hin_vocab_itos, hin_count)

tokenized.sort(key=lambda x: (len(x[0]), len(x[1])), reverse = True)


for i in tokenized:
    eng_sentence = CreateTrain(i[1], eng_vocab_stoi)
    hin_sentence = CreateTrain(i[0], hin_vocab_stoi)
    if len(eng_sentence)>2 and len(hin_sentence)>2:
        eng_train.append(eng_sentence)
        hin_train.append(hin_sentence)



*   Check vocabulary size and training data size


In [10]:
len(eng_vocab_itos), len(hin_vocab_itos), len(eng_train), len(hin_train)

(14964, 16360, 89194, 89194)



* A random sentence in training data after all its tokens are represented by corresponding indices


In [11]:
eng_train[434]

[2,
 5,
 20,
 171,
 2313,
 21,
 78,
 1748,
 21,
 5,
 54,
 1673,
 31,
 245,
 1494,
 63,
 33,
 1831,
 776,
 3]



*   device is set to cuda so that GPU can be used, if available
*   On doing so operations on tensors are done by the GPU (tensor is n dimensional arrays on which numeric compuatations can be done optimally, data passed to encoder will be of datatype, tensor)
*   Training data is split into batches with 32 sentences in one batch
*   Each column in the batch is an individual sentence with extra padding given at the end of shorter sentences so that each batch has same length equal to length of longest sentence in that batch (sorting by length of sentences was done earlier to reduce the need of padding)
*   Each batch is converted to tensor
*   At each time step, one row of the batch is sent, in the next time step the next row of the batch is sent, and so on. In simple words, single token of multiple sentences are being passed to encoder at the same time.



In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eng_batch = []
hin_batch = []

def CreateBatch(train, batch):
    k = 0
    while(k<len(train)):
        tmp_train = train[k:k+32]
        cur_batch = []

        for i in range(len(max(tmp_train, key=len))):
            cur_idx = []
            for j in range(len(tmp_train)):
                if len(tmp_train[j])>i:
                    cur_idx.append(tmp_train[j][i])
                else:
                    cur_idx.append(1)
            cur_batch.append(cur_idx)
        batch.append(torch.tensor(cur_batch))
        k = k+32

CreateBatch(eng_train, eng_batch)
CreateBatch(hin_train, hin_batch)



*   The size of last batch might not be equal to batch size, so remove that



In [14]:
eng_batch = eng_batch[:-1]
hin_batch = hin_batch[:-1]

len(eng_batch), len(hin_batch)



*   Check a random batch.


In [18]:
(hin_batch[-1])

tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2],
        [ 1020, 15092,   555,   102,  3549, 12444,  1679,   996,  1608,   194,
           212,  7290,   963,    41,  1025, 16279, 16284,  1920,  5278,   270,
             0,  3398,   146,     0,  4423, 11924, 15080,     0,  6129,  2861,
          7363,     0],
        [    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3]])

*   Initialize the specifications of the encoder and decoder for sequence to sequence model
*   The encoder and decoder input size must be specified as the vocabulary size of the source and target language (i.e. hindi and english vocabulary size) respectively --> input_size_encoder, input_size_decoder
*   This would be the size of one hot encoder in which each vector represents a unique token given by the index where it is 1 (rest all indices are 0)
*   Each token of a vocabulary is converted to a dense representation of words, that captures semantic information of the words in the vocabulary of the training set, called word embedding for which we have to specify the embedding size --> embedding_size
*   No pre-trained embedding is used
*   Basically hidden state and cell states are two output of LSTM cell which together constitute the context vector. This context vector (fixes size)captures the essence of the source sequences and is given to decoder along with target language (english) vocabulary to get translated sentences. Size of hidden state is specified --> hidden_state
*   Number of LSTM layers used for encoder and decoder is specified. If n, then n context vectors are obtained --> num_layers
*   Dropout is a regularization technique of ignoring random neurons during training to prevent overfitting. The probability of ignoring each neuron is specified --> dropout.
* [Reference used](https://towardsdatascience.com/a-comprehensive-guide-to-neural-machine-translation-using-seq2sequence-modelling-using-pytorch-41c9b84ba350)

In [19]:
input_size_encoder = len(hin_vocab_itos)
input_size_decoder = len(eng_vocab_itos)
embedding_size = 300
hidden_size = 1024
num_layers = 2
dropout = 0.5

* Now that all arguments passed to encoder have been specified, encoder LSTM is created.
* All the neural network functionality (including the various losses) are created using pytorch nn package. The nn package defines a set of Modules, which are roughly equivalent to neural network layers. It inputs and outputs tensors (which is why the batches given as input was converted to tensor earlier).
* Since custom encoder is created, nn.Module is subclassed and forward method defined. In forward method Tensor of input data is accepted and Tensor of output data is returned. Modules defined in the constructor as  well as arbitrary operators on Tensors can be used in forward method.
* Using the constructor, when a EncoderLSTM object is created, the attributes of the class EncoderLSTM are initialized, which are then used in forward method.   
* nn.Embedding takes a list of indices as input, and outputs the corresponding word embeddings.
* Embedding matrix is then given to the LSTM (nn.LSTM is used) which additionally uses previous time step's hidden state and cell state to produce hidden state and cell state that will be used by the LSTM in next time step.
* Dimension of all the tensors are given in code. (tensor.shape can be used to get the dimension of the tensor)
* [Reference used](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#pytorch-custom-nn-modules)


In [36]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()
    
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.dropout = nn.Dropout(p)
    self.tag = True ##
    
    # Dimension of embedding neural network --> (input_size, embedding_size)'s dimension
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Dimension of lstm cell neural network --> (hidden_size, cs)'s dimension
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    

  def forward(self, x):
    # input x is a vector of indices corresponding to tokens of current sentence (x.shape=sequence length, batch size)

    # embedding.shape = (sequence length, batch size, embedding_size) because now each x is also mapped to a D dimensional embedding.
    embedding = self.dropout(self.embedding(x))
    
    # outputs.shape = (sequence length, batch size, hidden_size)
    # hidden_state.shape = (num_layers, batch size, hidden_size)
    # cell_state.shape = (num_layers, batch size, hidden_size)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    # (hidden_state, cell_state) is the context vector (It will be used by decoder later)
    return hidden_state, cell_state

encoder_lstm = EncoderLSTM(input_size_encoder, embedding_size, hidden_size, num_layers, dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(16360, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


* The Context Vector from the Encoder LSTM is provided as the hidden state and cell state for the decoder’s first LSTM.
* The same arguments are passed to DecoderLSTM as passed to EncoderLSTM except for an extra argument, output_size. The output_size is same as input_size i.e. target (english) vocabulary size.
* The Encoder and Decoder are identical and code for Encoder and Decoder is the same, except that the prediction vector is also returned from the decoder. It has the probability of each token in the vocabulary being the correct prediction for all the tokens of the batch.


In [37]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()
    
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.output_size = output_size
    self.dropout = nn.Dropout(p)
    self.embedding = nn.Embedding(input_size, embedding_size)
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)


  def forward(self, x, hidden_state, cell_state):
    # Unlike Encoder in which all the token index in the same row of a batch is sent at a time, 
    # in decoder, each token is sent individually at each time step.

    x = x.unsqueeze(0)
    embedding = self.dropout(self.embedding(x))
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
    
    
    predictions = self.fc(outputs)

    # squeeze(0) removes dimension of size 1 
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

decoder_lstm = DecoderLSTM(input_size_decoder, embedding_size, hidden_size, num_layers, dropout, input_size_decoder).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(14964, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=14964, bias=True)
)


Create Sequence to Sequence model

In [45]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    batch_size = source.shape[1]

    target_len = target.shape[0]
    target_vocab_size = len(eng_vocab_stoi)
    
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    hidden_state, cell_state = self.Encoder_LSTM(source)
    
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    return outputs


Initialize model hyperparameter

In [46]:
# Hyperparameters
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = eng_vocab_stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

* The model is a sequence to sequence model with LSTM Encoder as first part that takes variable length input from source language and gives fixed length context vector that captures information about the sentences. The context vector is given to LSTM decoder which predicts a token at each time step till some terminating condition is reached, thus giving variable length output of target language.

In [47]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(16360, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(14964, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=14964, bias=True)
  )
)

In [49]:
epoch_loss = 0.0
best_loss = 999999
best_epoch = -1
# sentence1 = "कानून मानव जाति के लिए लागू होता है"

Train the model in small number of epoch each time and save the translation to prevent loss of work in case GPU gets disconnected.

In [50]:
num_epochs=10
sentence1 = "उससे बदतर हमारे पेशे ने कानून को जटिलता का चोगा पहना दिया है।"

for epoch in range(num_epochs):
  # #added
  # checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
  # save_checkpoint(checkpoint)


  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  # model.eval()
  # translated_sentence1 = translate_sentence(model, sentence1, device, max_length=30)
  # print(f"Translated example sentence 1: \n {translated_sentence1}")
  # ts1.append(translated_sentence1)

  model.train(True)
  for i in range(len(hin_batch)):
    # input = batch.src.to(device)
    # target = batch.trg.to(device)
    input = hin_batch[i].to(device)
    target = eng_batch[i].to(device)

    # Pass the input and target for model's forward method
    # print(i)
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()


Epoch - 1 / 10
saving

Epoch_Loss - 3.702443838119507

Epoch - 2 / 10
Epoch_Loss - 3.21035099029541

Epoch - 3 / 10
Epoch_Loss - 2.762840986251831

Epoch - 4 / 10
Epoch_Loss - 2.623439073562622

Epoch - 5 / 10
Epoch_Loss - 2.2964110374450684

Epoch - 6 / 10
Epoch_Loss - 2.1306848526000977

Epoch - 7 / 10
Epoch_Loss - 2.025114059448242

Epoch - 8 / 10
Epoch_Loss - 2.0840301513671875

Epoch - 9 / 10
Epoch_Loss - 2.130894660949707

Epoch - 10 / 10
Epoch_Loss - 1.9807953834533691



* Load the test hindi data from csv into a 2d array using python inbuilt csv reader.
* Preprocess it by normalizing it and removing punctuations like it was done for training data.

In [51]:
with open("drive/MyDrive/AssignmentNLP/hindistatements.csv", newline='') as f:
    reader = csv.reader(f)
    lines = list(reader)

normalizer = BaseNormalizer("hi", remove_nuktas=False)
translate_table = dict((ord(char), None) for char in string.punctuation)

test_sentences = []
for line in lines[1:]:
    test_sentences.append(normalizer.normalize(str(line[2]).translate(translate_table).strip()))

* Create translate_sentence function that translates a hindi text to english. 
* The function first tokenize input hindi sentence, append start and end tokens and create datatype, torch.LongTensor using index of tokens in the sentence.
* Then pass the tensor having index of tokens of current sentence to encoder and get context vector (hidden, cell states).
* Keep passing most recently obtained translated token to decoder, starting with start of string token. Of course also pass context vector obtained in previous step. Stop when end of string token is predicted by decoder or maximum sentence length is reached.
* Keep appending index of english translation from decoder to a list. Convert indices to english tokens using index to string dictionaries created earlier to create english translation of hindi input.
* Return the translated sentence.

In [ ]:
# def translate_sentence(model, sentence, device, max_length=30):
def translate_sentence(model, sentence, max_length=30):
    if type(sentence) == str:
        tokens = indic_tokenize.trivial_tokenize(sentence)
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, "<sos>")
    tokens.append("<eos>")

    text_to_indices=[]
    for token in tokens:
        if token in hin_vocab_stoi:
            text_to_indices.append(hin_vocab_stoi[token])
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)
    
    outputs = [eng_vocab_stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == hin_vocab_stoi["<eos>"]:
            break

    translated_sentence = [eng_vocab_itos[idx] for idx in outputs]
    return translated_sentence[1:]

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss): ##
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, '/content/checkpoint-NMT')
    torch.save(model.state_dict(),'/content/checkpoint-NMT-SD')

* Translate the hindi text to english using trained model by calling translate_sentence function.
* Remove unknown tokens for which no mapping has been found and end of string token.
* model.eval notifies the model that right now evaluation is taking place. This deactivates dropout.
* TreebankWordDetokenizer from nltk reverts the tokenized sentence to the original state.

In [52]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

model.eval()
pred_sentences = []

# for idx, i in enumerate(test_sentences):
for i test_sentences:
  model.eval()
  # translated_sentence = translate_sentence(model, i, device, max_length=50)
  translated_sentence = translate_sentence(model, i, max_length=50)
  pred_sentences.append(TreebankWordDetokenizer().detokenize(translated_sentence).replace('<ukn>','')[:-6].strip())

* See the predicted sentences.

In [53]:
(pred_sentences)

['who knows they know knows',
 'well travel your',
 'then they heard me',
 'inspirational',
 'doesnt',
 'and barbara we know a a a time time',
 'i never never anywhere',
 'and we were here here',
 'if we know i i i i',
 'shoot them them',
 'i call give obey',
 'because because is is',
 'he was a in',
 'accounts a',
 'we need need',
 'hed wasnt never leave',
 'so you you this',
 'i had a',
 'you dont',
 'a',
 'and its comes back back back',
 'i had a',
 'its too too beautiful',
 'because i know what',
 'noor  more our our and',
 'im so',
 'i dont believe this',
 'they knew we',
 'day another one and',
 'it doesnt be easy',
 'but since i i i i i i',
 'grunting grunting grunting',
 'i need a',
 'gunshots',
 'im me me',
 'give them one',
 'you know what what',
 'buddy abracabadbra right right right right right',
 'whistling',
 'and its a a a',
 'dont',
 'ill',
 'whered long',
 'maybe hes in in in',
 'if we ride a a a',
 'i was escaped the and and and and and and and and and and',
 'even na

* Write the translated english sentences in answer.txt which is submitted for the competition.

In [ ]:
with open("drive/MyDrive/AssignmentNLP/answer.txt", 'w') as f:
  for i in pred_sentences:
    f.write(i+'\n')

References:

* https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
* https://towardsdatascience.com/a-comprehensive-guide-to-neural-machine-translation-using-seq2sequence-modelling-using-pytorch-41c9b84ba350
* Numerous google search and question answer forum like stackoverflow